#**Corpus Translation**:
this notebook dedicated to developing our Arabic general advisor and strategy maker model. The goal of this project is to fine-tune our model using high-quality English data, which will be translated into Arabic, cleaned, and preprocessed for optimal performance.

We will begin by loading our English dataset, followed by translating it into Arabic using a transformer model to ensure linguistic accuracy and contextual relevance. After translation, we will conduct data cleaning, including the removal of unnecessary stop words, to enhance the dataset's quality and ensure it is suitable for training.

**Librairies Importation:**

In [ ]:
import pandas as pd
import numpy as np
import re
from google.colab import files
from transformers import MarianMTModel, MarianTokenizer
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [ ]:
uploaded = files.upload()
arabic_df_g = pd.read_csv('general.csv')

Saving general.csv to general.csv


In [ ]:
# Load CSV files
uploaded = files.upload()
arabic_df_s = pd.read_csv('strategie.csv')


Saving strategie.csv to strategie.csv


### Loading the translater Model and the Tokenizer from Hugging Face Librairie:

In [ ]:
import uuid
import json
import requests
import pandas as pd

# Azure Translation API settings
key = "9jtlp7kTr8AafwciHJGfmtzbl2wQAVs6LFehurUH6wgPhC8Uj7pTJQQJ99AJAC8vTInXJ3w3AAAbACOG9s8n"

endpoint = "https://api.cognitive.microsofttranslator.com/"
location = "westus2"

# Translation function
def translate_df_to_arabic(df):
    # Define API parameters and headers
    path = '/translate'
    constructed_url = endpoint + path
    params = {
        'api-version': '3.0',
        'from': 'en',
        'to': ['ar']
    }
    headers = {
        'Ocp-Apim-Subscription-Key': key,
        'Ocp-Apim-Subscription-Region': location,
        'Content-type': 'application/json',
        'X-ClientTraceId': str(uuid.uuid4())
    }

    # Function to translate a single text
    def translate_text(text):
        body = [{'text': text}]
        response = requests.post(constructed_url, params=params, headers=headers, json=body)
        if response.status_code == 200:
            return response.json()[0]['translations'][0]['text']
        else:
            print(f"Request failed with status code {response.status_code}: {response.text}")
            return text  # Return the original text if translation fails

    # Apply translation to each entry in the "Question" and "Answer" columns
    df_arabic = df.copy()
    df_arabic['Question'] = df['Question'].apply(translate_text)
    df_arabic['Answer'] = df['Answer'].apply(translate_text)

    return df_arabic





In [ ]:
# Translate the general advise df to Arabic
arabic_df_g  = translate_df_to_arabic(arabic_df_g )
display(arabic_df_g)

,Question,Answer
0,ما هي الجوانب الرئيسية للتمويل الشخصي؟,يتضمن التمويل الشخصي إدارة دخلك ونفقاتك ومدخرا...
1,كيف يمكنني إنشاء ميزانية جيدة؟,ابدأ بسرد جميع مصادر الدخل وتصنيف نفقاتك. خصص ...
2,ما هي العادات المالية الجيدة التي يجب تطويرها؟,تشمل العادات الجيدة تتبع إنفاقك ، والادخار باس...
3,لماذا صندوق الطوارئ مهم؟,يساعدك صندوق الطوارئ على تغطية النفقات غير الم...
4,كيف يمكنني تحسين درجة الائتمان الخاصة بي؟,لتحسين درجة الائتمان الخاصة بك ، ادفع الفواتير...
...,...,...
97,ما هي أفضل طريقة لبدء الاستثمار؟,ابدأ بتحديد مدى تحملك للمخاطر المالية وأهدافك....
98,كيف يمكنني التعامل مع الادخار للتقاعد؟,وفر 15-20٪ من دخلك سنويا إن أمكن. استفد من الم...
99,كيف يمكنني تحسين درجة الائتمان المالية الخاصة بي؟,الحفاظ على تاريخ مالي قوي من خلال دفع الفواتير...
100,هل الأفضل شراء أو استئجار سيارة ماليا؟,من منظور التمويل ، غالبا ما يكون للشراء مدخرات...


In [ ]:
# Translate the general advise df to Arabic
arabic_df_s = translate_df_to_arabic(arabic_df_s)
display(arabic_df_s )

,Question,Answer
0,أنا مستقل أبلغ من العمر 27 عاما في مجال التسوي...,مع دخل مستقل مستقر ، ابدأ بإنشاء شبكة أمان من ...
1,أنا صاحب شركة صغيرة أبلغ من العمر 32 عاما ويبل...,ضع في اعتبارك SEP IRA أو Solo 401 (k) للأفراد ...
2,أنا مؤسس شركة ناشئة في مجال التكنولوجيا أبلغ م...,من الضروري تحقيق التوازن بين النمو والمخاطر. خ...
3,أنا رائد أعمال أبلغ من العمر 45 عاما ولدي دخل ...,تعظيم المساهمات في حسابات التقاعد مثل 401 (k) ...
4,بصفتي صاحب شركة تجارة إلكترونية يبلغ من العمر ...,إنشاء احتياطي نقدي لإدارة الدخل المتقلب ، من ا...
...,...,...
95,أنا مهندس برمجيات أبلغ من العمر 32 عاما وأكسب ...,بصفتك مهندس برمجيات يبلغ دخله الشهري حوالي 137...
96,أنا صاحب شاحنة طعام أبلغ من العمر 33 عاما وأكس...,بصفتك مالكا لشاحنة طعام يبلغ دخلها الشهري حوال...
97,أنا مدرب يوغا أبلغ من العمر 34 عاما وأكسب حوال...,بصفتك مدربا لليوغا بدخل شهري يبلغ حوالي 14000 ...
98,أنا منشئ محتوى أبلغ من العمر 35 عاما وأكسب حوا...,بصفتك منشئ محتوى بدخل شهري يبلغ حوالي 14125 دو...


### Corpus Preprocessing :

In [ ]:
# Download NLTK resources (if not already done)
nltk.download('punkt')
nltk.download('stopwords')

# Load Arabic stop words
arabic_stop_words = set(stopwords.words('arabic'))

def remove_nulls_and_duplicates(df):
    null_count = df.isnull().sum().sum()
    duplicate_count = df.duplicated().sum()

    if null_count > 0:
        df = df.dropna()
    if duplicate_count > 0:
        df = df.drop_duplicates()

    return df

def remove_arabic_stop_words(df):
    for column in ["Question", "Answer"]:
        if column in df.columns:
            df[column] = df[column].apply(lambda x: ' '.join(
                [word for word in word_tokenize(x) if word not in arabic_stop_words]
            ) if isinstance(x, str) else x)
    return df

# Function to limit text length
def limit_text_length(df, max_length=5000):
    for column in ["Question", "Answer"]:
        if column in df.columns:
            df[column] = df[column].apply(lambda x: x[:max_length] if isinstance(x, str) else x)
    return df

# Combined preprocessing function
def preprocess_dataframe(df):
    df = remove_nulls_and_duplicates(df)
    df = remove_arabic_stop_words(df)
    df = limit_text_length(df)
    return df

arabic_df_g_preprocessed = preprocess_dataframe(arabic_df_g)
arabic_df_s_preprocessed = preprocess_dataframe(arabic_df_s)




[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [21]:
# Display preprocessed DataFrames
print("The General Model Data \n")
display(arabic_df_g_preprocessed)
print("The Strategie Model Data \n")
display(arabic_df_s_preprocessed)

The General Model Data 



,Question,Answer
0,الجوانب الرئيسية للتمويل الشخصي؟,يتضمن التمويل الشخصي إدارة دخلك ونفقاتك ومدخرا...
1,يمكنني إنشاء ميزانية جيدة؟,ابدأ بسرد مصادر الدخل وتصنيف نفقاتك . خصص جزءا...
2,العادات المالية الجيدة يجب تطويرها؟,تشمل العادات الجيدة تتبع إنفاقك ، والادخار باس...
3,لماذا صندوق الطوارئ مهم؟,يساعدك صندوق الطوارئ تغطية النفقات المتوقعة ، ...
4,يمكنني تحسين درجة الائتمان الخاصة بي؟,لتحسين درجة الائتمان الخاصة ، ادفع الفواتير ال...
...,...,...
97,أفضل طريقة لبدء الاستثمار؟,ابدأ بتحديد مدى تحملك للمخاطر المالية وأهدافك ...
98,يمكنني التعامل الادخار للتقاعد؟,وفر 15-20٪ دخلك سنويا أمكن . استفد المضاعفة خل...
99,يمكنني تحسين درجة الائتمان المالية الخاصة بي؟,الحفاظ تاريخ مالي قوي خلال دفع الفواتير الوقت ...
100,الأفضل شراء استئجار سيارة ماليا؟,منظور التمويل ، يكون للشراء مدخرات طويلة الأجل...


The Strategie Model Data 



,Question,Answer
0,مستقل أبلغ العمر 27 عاما مجال التسويق الرقمي و...,دخل مستقل مستقر ، ابدأ بإنشاء شبكة أمان 3-6 أش...
1,صاحب شركة صغيرة أبلغ العمر 32 عاما ويبلغ دخلي ...,ضع اعتبارك SEP IRA Solo 401 ( k ) للأفراد العا...
2,مؤسس شركة ناشئة مجال التكنولوجيا أبلغ العمر 24...,الضروري تحقيق التوازن النمو والمخاطر . خصص جزء...
3,رائد أعمال أبلغ العمر 45 عاما ولدي دخل ثابت شر...,تعظيم المساهمات حسابات التقاعد 401 ( k ) SEP I...
4,بصفتي صاحب شركة تجارة إلكترونية يبلغ العمر 30 ...,إنشاء احتياطي نقدي لإدارة الدخل المتقلب ، النا...
...,...,...
95,مهندس برمجيات أبلغ العمر 32 عاما وأكسب حوالي 1...,بصفتك مهندس برمجيات يبلغ دخله الشهري حوالي 137...
96,صاحب شاحنة طعام أبلغ العمر 33 عاما وأكسب حوالي...,بصفتك مالكا لشاحنة طعام يبلغ دخلها الشهري حوال...
97,مدرب يوغا أبلغ العمر 34 عاما وأكسب حوالي 14000...,بصفتك مدربا لليوغا بدخل شهري يبلغ حوالي 14000 ...
98,منشئ محتوى أبلغ العمر 35 عاما وأكسب حوالي 1412...,بصفتك منشئ محتوى بدخل شهري يبلغ حوالي 14125 دو...


Data Downloading in csv format for fine Tuning :

In [ ]:
arabic_df_g_preprocessed.to_csv("arabic_df_g_preprocessed.csv", index=False)
arabic_df_s_preprocessed.to_csv("arabic_df_s_preprocessed.csv", index=False)
# Download the file
def download_csv(nom):
  files.download(nom)
download_csv("arabic_df_g_preprocessed.csv")
download_csv("arabic_df_s_preprocessed.csv")




<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>